# **Mel-Frequency Cepstral Coefficients (MFCCs)**
MFCCs are coefficients that collectively make up an MFC. They are derived from the Fourier transform of a signal and are widely used in speech and audio processing.

In [ ]:
%pip install librosa numpy matplotlib

In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import logging

from IPython.display import Audio, display

In [ ]:
# Adjust sys.path to include the root directory
logging.info("Adjusting sys.path to include the root directory")
root_dir = os.path.abspath(os.path.join(os.getcwd(), '../../'))
if root_dir not in sys.path:
    sys.path.append(root_dir)
logging.info(f"sys.path adjusted: {sys.path}")

In [ ]:
# Local Imports & Parameters
logging.info("Local configurations and parameters imported")
from config.config import audio_config, output_config
from config.parameters import *

from config.logging import setup_logging
# from config.utils import ensure_directory

# Plot configurations
from config.matplotlib_plots import configure_plot
from config.matplotlib_plots import create_custom_colormap
logging.info("Local configurations and parameters imported")

In [ ]:
# Set up logging for this notebook
logging.info("Setting up logging for the notebook")
notebook_path = os.path.join(os.getcwd(), 'MFCCs.ipynb')
setup_logging(notebook_path)
logging.info("Logging set up complete")

In [ ]:
# Set the audio file to analyse
logging.info("Setting the audio file to analyse")
audio_file_key = AUDIO_FILE_SAX_A3
audio_file_path = audio_config.get_audio_file(audio_file_key)

logging.info(f"Audio file set to: {audio_file_path}")

In [ ]:
def analyse_audio(audio_file_path):
    """
    Load an audio file, calculate its MFCCs, and plot the MFCCs.

    Args:
        audio_file_path (str): Path to the audio file.

    Returns:
        tuple: (y, sr, mfccs)
            y: Audio time series
            sr: Sampling rate
            mfccs: MFCCs of the audio
    """
    try:
        logging.info(f"Loading audio file from: {audio_file_path}")
        y, sr = librosa.load(audio_file_path, sr=None)

        logging.info("Calculating MFCCs")
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

        return y, sr, mfccs

    except FileNotFoundError:
        logging.error(f"File not found: {audio_file_path}")
    except Exception as e:
        logging.error(f"An error occurred: {e}")

In [ ]:
try:
    y, sr, mfccs = analyse_audio(audio_file_path)
    logging.info(f"Successfully analysed audio file {audio_file_path}")

except Exception as e:
    logging.error(f"Error analysing audio file {audio_file_path}: {e}")
    raise

# Display the audio
audio_display = Audio(audio_file_path)
audio_display

In [ ]:
# Function to ensure the directory exists
def ensure_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)


In [ ]:
# Plot the MFCCs
logging.info("Plotting the MFCCs")
fig, ax = plt.subplots(figsize=(14, 5))

# Extract the file name from the file path
file_name = os.path.basename(audio_file_path)
file_base_name, _ = os.path.splitext(file_name)

# Plot the MFCCs
img = librosa.display.specshow(mfccs, x_axis='time', ax=ax, cmap=custom_cmap)
cbar = fig.colorbar(img, ax=ax)

# Set color for the color bar labels and spine
cbar.ax.yaxis.set_tick_params(color=SPINE_COLOR)
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color=SPINE_COLOR)
cbar.ax.yaxis.label.set_color(SPINE_COLOR)
cbar.outline.set_edgecolor(SPINE_COLOR)
for spine in cbar.ax.spines.values():
    spine.set_edgecolor(SPINE_COLOR)

# Configure the plot
configure_plot(ax, title=file_name, subtitle="MFCCs")

# Ensure the output directory exists
output_directory = output_config.get_output_directory("frequency_domain")
ensure_directory(output_directory)

# Save the plot
output_path = os.path.join(output_directory, f'MFCC_{file_base_name}.png')
plt.savefig(output_path, facecolor=BACKGROUND_COLOR)

# Log the saved plot location
logging.info(f"Plot saved to: {output_path}")

# Show the plot
plt.show()